# Replay  데이터 증강

## 사용법
---
* 설정은 아래 SETTING 주석 처리한 부분에서 원하시는 설정을 진행해 주시면 됩니다.
* 추출한 데이터는 Pro_Cybersecurity_Car_Hacking_D_training-0.csv, Pro_Cybersecurity_Car_Hacking_S_training-0.csv  
16진수 정상데이터에서 추출해야하므로 반드시 두 파일을 origin_fname에 설정하세요.
위의 두 파일은 TimeStamp 소숫점 자릿값이 잘려있기 때문에 예선 대회 원본 데이터 0번을 함께 originRaw에 입력해야합니다.

세트별 구간수는 %단위이며 리플레이가 해당범위안에서 랜덤으로 시작합니다.<br>   **주의: 이는 리플레이의 구간 길이가 아니며 리플레이가 랜덤으로 처음 시작할 구간을 설정하는 작업입니다**
<br>
리플레이의 attackRow는 한 공격의 리플레이 row수를 설정하는 것이며 이는 리플레이 구간의 row수가 아닌 순수 Subclass가 Replay로 분류되는 row의 수를 입력합니다.
<br>
리플레이 삽입하는데 시간이 좀 오래걸립니다....

<br><br><br>
<br>
### Replay 분석 및 계획
---
**정상데이터를 어떻게 잘라서 가져올지?**  
**Replay를 얼마나 간격을 두고 row를 삽입할지?**



ID 130 뒤에는 항상 140이옴  
학습데이터 (정상, 공격상황 모두) 와 제출데이터 모두 동일하게 적용됨  
130세트의 수 길이가 크게 차이나지 않음 (20-30내외)  
최소 기준점을 ID 130으로 잡음

* 130 패턴 전체를 한 세트로 잡아야 좋을지? <br> ex) Data_7의 반복되는 패턴 0A\~0F\~0\~9 <br> 공격 데이터에서는 Data_7 은 0E로 시작하는 것만은 아님

### 공격세트 구성안
> * 세트 구성 ID 130으로 패턴 시작을 랜덤하게 하여 한 세트를 구성 <br>
* ID 130 세트 시작점을 랜덤하게 구성
* ID 상관없이 랜덤하게 세트 구성   - 가장 유력한 정책으로-

time interval 을 변화를 주어 생성 예정


### 구성 계획
---
1. 학습데이터에서 세트 추출
2. 충분한 세트 수 추출 후 여러 세트로 구성한 공격 세트 제작
3. 공격세트를 주입할 원본 데이터에 삽입

### 공격데이터 삽입
1~11 사이의 랜덤한 간격으로 한 row씩 삽입

In [1]:
import pandas as pd
import time
from random import *
from tqdm import notebook

pd.options.display.float_format = '{:.7f}'.format

In [21]:
attackArea = list()
attackRow = list()


######## SETTING ##########################################################################
origin_fname = "Pro_Cybersecurity_Car_Hacking_D_training-0.csv" # Replay 추출할 파일
target_fname = "Pro_Cybersecurity_Car_Hacking_D_training-0.csv" # Replay 주입할 파일

#timestamp 소수점 잘려서 넣은 코드
originRAW_fname = "Cybersecurity_Car_Hacking_D_training-0.csv" #추출할 데이터의 온전한 타임스탬프가 들어있는 파일 
targetRAW_fname = "Cybersecurity_Car_Hacking_D_training-0.csv" # 주입할 데이터의 온전한 타임스탬프가 들어있는 파일

save_fname = "JeunGangData.csv" # 증강된 데이터 저장이름 입력



# ------- 공격 세트 수 -------
# setAttackNum = 공격할 리플레이 세트 수
setAttackNum = 3 #예시

# ------- 공격 세트 별 시작구간 설정 (공격 세트 수 맞춰야함 안맞출시 런타임 오류 발생!)--------
# attackArea.append([처음 등장할 시작%, 처음 등장할 끝%])
attackArea.append([1,2]) #예시 3개
attackArea.append([4,5])
attackArea.append([7,8])

# ------- 한 리플레이 공격의 row 수 설정 (공격 세트 수 맞춰야함 안맞출시 런타임 오류 발생!)--------
# attackRow.append(한공격의 row수)
attackRow.append(4869) #예시 3개
attackRow.append(5309)
attackRow.append(5949)



#########################################################################################


assert setAttackNum == len(attackArea), "공격 세트 수와 구간 수를 맞춰주세요:)"
assert setAttackNum == len(attackRow), "공격 세트 수와 공격 row수를 맞춰주세요:)"

In [3]:
data = pd.read_csv(origin_fname)
target = pd.read_csv(target_fname)

originTimestampDf = pd.read_csv(originRAW_fname)
targetTimestampDf = pd.read_csv(targetRAW_fname)

data["Timestamp"] = originTimestampDf["Timestamp"]
target["Timestamp"] = targetTimestampDf["Timestamp"]



# 랜덤구간 데이터 추출 (사용할 알고리즘)

In [4]:
# 정상데이터에서 랜덤한 인덱스 공격세트 갯수만큼 추출후 리스트에 저장
# 리스트 순회하면서 원하는 구간의 설정한 row의 갯수만큼 공격세트에 저장
# 공격리스트 반환

def MakeAttackSet3(df, attackRow, setNum):
    
    startIndex = 0
    endIndex = len(df)
#    indexList = list()
    attackList = list()
    
    for i in notebook.tqdm(range(setNum), desc = "공격 세트 생성중"):
        index = randrange(len(df)-attackRow[i]+1)
#         indexList.append(index)
        
        attackSet = df.iloc[index:index + attackRow[i]]
#         for j in notebook.tqdm(range(attackRow[i]), desc = "한 공격 세트 생성중"):
#             attackSet.append(df.iloc[[index]])
#             index += 1
        
        attackSet["Class"] = "Attack"
        attackSet["SubClass"] = "Replay"
        attackList.append(attackSet)
            
    return attackList
    
    

In [5]:
attackList = MakeAttackSet3(data, attackRow, setAttackNum)

<ipython-input-4-66612e42e13c>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  attackSet["Class"] = "Attack"
<ipython-input-4-66612e42e13c>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  attackSet["SubClass"] = "Replay"


## ~Replay 세트 시작 인덱스 추출~

In [6]:
# # 추출 필요 parameter: 원본추출데이터, 
# # 130만날 때 마다 리스트 인덱스 추출
# # 130의 인덱스 리스트 반환


# def GetRepIdx(df):
    
#     startIndex = 0
#     endIndex = len(df)
#     cur = startIndex
    
#     setIndex = 0
#     repIdxList = list()
    
    
#     for i in notebook.tqdm(range(endIndex)):
#         if str(df.loc[cur, "Arbitration_ID"]) == "130": # 처음 찾은 130 1세트 시작
#             #replayList = list()
#             repIdxList.append(cur)
#             cur += 1
#         else:
#             cur += 1
        
#     repIdxList.pop()
#     return repIdxList
        

In [7]:
# testIdxList = GetRepIdx(data)

# 공격세트 삽입

In [8]:
# 삽입 필요 parameter: 타겟데이터, 130 사이클 인덱스 리스트, 공격 데이터 리스트
# 공격 갯수 만큼 공격세트를 만들어(공격데이터 함수 이용) 공격리스트에 저장
# 테스트 세트 수를 순회하며 랜덤한 지점에 한세트 씩 삽입
# 공격리스트 원소 순회하면서 한 공격 세트 안의 row를 순차적으로 정상데이터와의 1~11 간격으로 랜덤하게 삽입


def InsertAttackData(targetDf, attackList):
    result = targetDf
    result["SubClass"] = "Normal"
    
    for i in notebook.tqdm(range(len(attackList)), desc = "전체 공격 세트 삽입"): #테스트 세트 하나씩 삽입준비 (공격세트는 랜덤으로 리스트에 들어있음)
        
        
        insertStart = int(len(targetDf) / 100 * attackArea[i][0])
        insertEnd = int(len(targetDf) / 100 * attackArea[i][1])
        
        randindex = randrange(insertStart, insertEnd) # 주어진 구간에서 랜덤한 위치에
        
        cur = randindex # 순회하는 iter
        
        for j in notebook.tqdm(range(len(attackList[i])), desc = "공격 세트 한개 삽입"):
            
            head = result.iloc[:cur]
            tail = result.iloc[cur:]
            
            timediff = head["Timestamp"] - tail["Timestamp"]
            addtoTimestamp = uniform(0,timediff)
            
            
            result = pd.concat([head, attackList[i].iloc[[j]], tail])
            
            
            #result.iloc[cur]["Timestamp"] = result.iloc[cur-1]["Timestamp"] + timediff  #settingwithcopywaring 발생!
            
            resultTS = (result.iloc[cur-1]["Timestamp"] + timediff).copy() #settingwithcopywaring 방지
            result.iloc[cur]["Timestamp"]
            
            
            cur += randrange(1,11) ############## 1~11까지 간격 조정 ##############
            
            # 타임 인터벌은 배수로 일단 순서에 상관없이 주입하는 코드를 작성할 것!
    
    return result
            
        
    

In [9]:
testResult = InsertAttackData(target, attackList)

# 결과

In [10]:
testResult

,Timestamp,Arbitration_ID,DLC,Data_1,Data_2,Data_3,Data_4,Data_5,Data_6,Data_7,Data_8,Class,SubClass
0,1597707827.0525911,260,8,6,25,5,30,FF,CF,71,55,Normal,Normal
1,1597707827.0539801,329,8,4A,C5,7E,8C,31,2D,1,10,Normal,Normal
2,1597707827.0546699,38D,8,0,0,49,0,90,7F,FE,1,Normal,Normal
3,1597707827.0549040,420,8,50,1E,0,C8,FC,4F,6A,0,Normal,Normal
4,1597707827.0551400,421,8,FE,7,0,FF,E3,7F,0,52,Normal,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...
179341,1597707901.7066751,391,8,0,0,0,0,0,0,8,EB,Normal,Normal
179342,1597707901.7069051,260,8,6,39,1A,30,FF,D1,A1,63,Normal,Normal
179343,1597707901.7071440,421,8,FE,7,0,FF,E3,7F,0,9E,Normal,Normal
179344,1597707901.7073779,130,8,94,8E,F0,81,0,0,0B,AC,Normal,Normal


In [11]:
resultRep = testResult[testResult["SubClass"] == "Replay"]
resultRep

,Timestamp,Arbitration_ID,DLC,Data_1,Data_2,Data_3,Data_4,Data_5,Data_6,Data_7,Data_8,Class,SubClass
62677,1597707853.1265669,140,8,A0,81,0,6F,20,0,4,D0,Attack,Replay
62678,1597707853.1292961,340,8,0,0,0,24,26,1,3D,30,Attack,Replay
62679,1597707853.1304390,164,4,0,8,1A,B4,NaN,NaN,NaN,NaN,Attack,Replay
62680,1597707853.1306720,251,8,90,3,9,1C,0,A9,27,80,Attack,Replay
62681,1597707853.1308751,2B0,6,F8,FC,5,7,F9,12,NaN,NaN,Attack,Replay
...,...,...,...,...,...,...,...,...,...,...,...,...,...
111731,1597707873.5543900,5A6,8,0,0,0,0,0,0,0,0,Attack,Replay
111732,1597707873.5556722,47F,8,0,1,FF,FA,0,78,0,A7,Attack,Replay
111733,1597707873.5559099,153,8,20,80,10,FF,0,FF,A0,4E,Attack,Replay
111734,1597707873.5561450,220,8,11,44,7D,0,0,A3,10,9C,Attack,Replay


In [12]:
#인덱스 초기화
testResult.reset_index(inplace = True)
testResult.drop("index", axis = 1, inplace = True)
testResult

,Timestamp,Arbitration_ID,DLC,Data_1,Data_2,Data_3,Data_4,Data_5,Data_6,Data_7,Data_8,Class,SubClass
0,1597707827.0525911,260,8,6,25,5,30,FF,CF,71,55,Normal,Normal
1,1597707827.0539801,329,8,4A,C5,7E,8C,31,2D,1,10,Normal,Normal
2,1597707827.0546699,38D,8,0,0,49,0,90,7F,FE,1,Normal,Normal
3,1597707827.0549040,420,8,50,1E,0,C8,FC,4F,6A,0,Normal,Normal
4,1597707827.0551400,421,8,FE,7,0,FF,E3,7F,0,52,Normal,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195468,1597707901.7066751,391,8,0,0,0,0,0,0,8,EB,Normal,Normal
195469,1597707901.7069051,260,8,6,39,1A,30,FF,D1,A1,63,Normal,Normal
195470,1597707901.7071440,421,8,FE,7,0,FF,E3,7F,0,9E,Normal,Normal
195471,1597707901.7073779,130,8,94,8E,F0,81,0,0,0B,AC,Normal,Normal


In [13]:
resultRep = testResult[testResult["Class"] == "Attack"]
resultRep

,Timestamp,Arbitration_ID,DLC,Data_1,Data_2,Data_3,Data_4,Data_5,Data_6,Data_7,Data_8,Class,SubClass
2281,1597707853.1265669,140,8,A0,81,0,6F,20,0,4,D0,Attack,Replay
2287,1597707853.1292961,340,8,0,0,0,24,26,1,3D,30,Attack,Replay
2289,1597707853.1304390,164,4,0,8,1A,B4,NaN,NaN,NaN,NaN,Attack,Replay
2299,1597707853.1306720,251,8,90,3,9,1C,0,A9,27,80,Attack,Replay
2303,1597707853.1308751,2B0,6,F8,FC,5,7,F9,12,NaN,NaN,Attack,Replay
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45766,1597707873.5543900,5A6,8,0,0,0,0,0,0,0,0,Attack,Replay
45772,1597707873.5556722,47F,8,0,1,FF,FA,0,78,0,A7,Attack,Replay
45779,1597707873.5559099,153,8,20,80,10,FF,0,FF,A0,4E,Attack,Replay
45788,1597707873.5561450,220,8,11,44,7D,0,0,A3,10,9C,Attack,Replay


# 저장

In [22]:
testResult.to_csv(save_fname)

# 연습장
---
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>















In [14]:
# print(test)
# #print(data.iloc[3])
# testList = list()
# testList.append(data.iloc[[3]])
# testList.append(data.iloc[[4]])
# testList.append(data.iloc[[4]])
# #pd.DataFrame(testList)
# testResultList = pd.concat([testList[0], testList[1], testList[2]])
# testResultList

In [15]:
# data.iloc[randrange(len(testIdxList))]
# target

## Replay 공격 세트 추출 및 제작 -> 폐기(긴 실행시간)

In [16]:
# # 추출 필요 parameter: 원본추출데이터, 
# # 130만날 때 마다 잘라서 리스트에 저장 -> 세트 수 리스트 완성
# # 리스트 반환


# def GetReplay(df):
    
#     startIndex = 0
#     endIndex = len(df)
#     cur = startIndex
    
#     setIndex = 0
#     replayList = list()
    
    
#     for i in tqdm(range(endIndex)):
#         if str(df.loc[cur, "Arbitration_ID"]) == "130": # 처음 찾은 130 1세트 시작
#             #replayList = list()
#             replayList.append(df.iloc[[cur]])
#             cur += 1
#             while df.loc[cur, "Arbitration_ID"] != "130":
#                 replayList[setIndex] = pd.concat([replayList[setIndex], df.iloc[[cur]]]) # 130 한 사이클 한 세트 구성
#                 cur += 1
#             setIndex += 1
#             # return replayList
#         else:
#             cur += 1
        
#     return replayList
        

## ~1. 공격 세트 제작 - 시작인덱스를 이용하여 랜덤하게 배치하여 세트제작~

In [17]:
# # 세트 제작 필요 parmeter: 데이터, 시작인덱스리스트, 공격 구성 세트 수
# # 시작인덱스리스트 길이에서 난수를 뽑아 세트수 구성하여 공격세트 구성
# # 랜덤한 130세트를 선정하여 공격세트를 구성

# def MakeAttackSet(df, idxList, setNum):
#     endIndex = len(df)
    
#     randomNum = randrange(len(idxList))
#     attackSet = list() #각 세트를 리스트에 저장
#     attackSetidx = 0 #각 세트를 저장한 리스트의 인덱스
    
#     for i in notebook.tqdm(range(setNum), desc = "세트 리스트 생성"):
#         rdmidx = randrange(len(idxList)) # 랜덤한 인덱스 사용
#         cur = idxList[rdmidx]
# #         print("현재 %d 인덱스에서 작업중" %rdmidx)
#         if str(df.loc[cur, "Arbitration_ID"]) == "130":
#             attackSet.append(df.iloc[[cur]])
#             cur += 1
#             count = 1 ############130 한세트의 갯수############
#             while str(df.loc[cur, "Arbitration_ID"]) != "130":
                    
#                 tmp = df.iloc[[cur]]
#                 attackSet[attackSetidx] = pd.concat([attackSet[attackSetidx], tmp])
#                 cur += 1
                
#                 count += 1
            
                
#             print("set[%d] 길이: " %(attackSetidx) ,count)
#             attackSetidx += 1 # 다음 세트로
            
#     # 세트를 저장한 리스트를 이용하여 공격 세트 한개 제작
#     for i in notebook.tqdm(range(1, setNum), desc = "세트 리스트를 이용하여 공격 세트 만드는 중"):
#         attackSet[0] = pd.concat([attackSet[0], attackSet[i]])
    
#     resultSet = attackSet[0].copy()
#     resultSet["Class"] = "Attack"
#     resultSet["SubClass"] = "Replay"
#     return resultSet

## ~2. 130 사이클 순서는 그대로 두되 시작점만 다르게 공격세트를 구성~

In [18]:
# # 세트 제작 필요 parmeter: 데이터, 시작인덱스리스트, 공격 구성 세트 수
# # 시작인덱스리스트 길이에서 난수를 뽑아 세트수 구성하여 공격세트 구성
# # 랜덤 시작점을 두고 순서는 정상데이터와 일치하여 공격세트 구성


# def MakeAttackSet2(df, idxList, setNum):
#     endIndex = len(df)
    
#     randomNum = randrange(len(idxList))
#     attackSet = list() #각 세트를 리스트에 저장
#     attackSetidx = 0 #각 세트를 저장한 리스트의 인덱스
    
#     rdmidx = randrange(len(idxList))
#     rdmStartidx = idxList[rdmidx] # 랜덤하게 시작할 인덱스 선정
#     cur = rdmStartidx
    
#     for i in notebook.tqdm(range(setNum), desc = "세트 리스트 생성"):
# #         rdmidx = randrange(len(idxList))
# #         cur = idxList[rdmidx]

# #         print("현재 %d 인덱스에서 작업중" %rdmidx)
#         if str(df.loc[cur, "Arbitration_ID"]) == "130":
#             attackSet.append(df.iloc[[cur]])
#             cur += 1
#             count = 1 ############130 한세트의 갯수############
#             while str(df.loc[cur, "Arbitration_ID"]) != "130":
                    
#                 tmp = df.iloc[[cur]]
#                 attackSet[attackSetidx] = pd.concat([attackSet[attackSetidx], tmp])
#                 cur += 1
                
#                 count += 1
            

#             print("set[%d] 길이: " %(attackSetidx) ,count)
#             attackSetidx += 1 # 다음 세트로
            
#     # 세트를 저장한 리스트를 이용하여 공격 세트 한개 제작
#     for i in notebook.tqdm(range(1, setNum), desc = "세트 리스트를 이용하여 공격 세트 만드는 중"):
#         attackSet[0] = pd.concat([attackSet[0], attackSet[i]])
    
#     resultSet = attackSet[0].copy()
#     resultSet["Class"] = "Attack"
#     resultSet["SubClass"] = "Replay"
#     return resultSet

# ~공격 데이터 리스트 생성~

In [19]:
# # 공격데이터에서 130 세트 몇 사이클인지 확인할것!
# # 다음 130 사이클이 시작할 타임에 랜덤하게 삽입

# testAttackSet = list()

# for i in notebook.tqdm(range(setAttackNum), desc = "전체 생성 진행률"): # 미리 설정한 공격 세트 수 만큼 생성
    
#     ### 임시 130 세트 수
    
#     testAttackSet.append(MakeAttackSet2(data, testIdxList, 15))
    
#     ### 실제에서 사용할 130 세트 수
    
#     ####testAttackSet.append(MakeAttackSet(data, testIdxList, randrange(230, 260))) #몇 사이클인지 확인후 수정 <대략 5000~6000 row로 구성>

#     ###